# 0. 準備

In [ ]:
import scipy 
from scipy.linalg import solve_triangular
from scipy.linalg.lapack import dpotrf, dtrtri, dpotrs, dpotri
import numpy as np
import pandas as pd

import sys
sys.path.append('../')
from linalg_util import *

In [ ]:
import gp_util
import importlib
importlib.reload(gp_util)